In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y/10
    

In [3]:
label = "1D_SODE_tanh_scaled"
loss_thresh = 0.005

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0/10.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 

    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a/50
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)

    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 10
max_iter = 100

N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

for reps in range(max_reps):
    
    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    
    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 
    
    
    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-5, 
                              tolerance_change = 1e-5, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')
   

    start_time = time.time()
    train_model(max_iter,reps)

    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
 
    
    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "label": label, "Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 0.036655657 Test MSE 3.8837616761265634 Test RE 1.00459004105692
1 Train Loss 0.024073191 Test MSE 3.8470622394994995 Test RE 0.9998323601080702
2 Train Loss 0.023913605 Test MSE 3.842426539910834 Test RE 0.9992297808825481
3 Train Loss 0.02390569 Test MSE 3.8420818159495105 Test RE 0.9

98 Train Loss 0.02382048 Test MSE 3.83941368269108 Test RE 0.9988379546495351
99 Train Loss 0.02382048 Test MSE 3.83941368269108 Test RE 0.9988379546495351
Training time: 6.78
Training time: 6.78
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 0.026567467 Test MSE 3.8492728941508454 Test RE 1.0001195879308467
1 Train Loss 0.024157144

92 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
93 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
94 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
95 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
96 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
97 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
98 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
99 Train Loss 0.023816258 Test MSE 3.8391376291908363 Test RE 0.9988020458242806
Training time: 6.76
Training time: 6.76
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_fea

87 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
88 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
89 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
90 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
91 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
92 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
93 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
94 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
95 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
96 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
97 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
98 Train Loss 0.023842957 Test MSE 3.8411615038408486 Test RE 0.9990652799141777
99 Train Loss 0.023842957 Te

87 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
88 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
89 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
90 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
91 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
92 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
93 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
94 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
95 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
96 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
97 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
98 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.999374500614583
99 Train Loss 0.02392435 Test MSE 3.843539627647774 Test RE 0.99

...

74 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
75 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
76 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
77 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
78 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
79 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
80 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
81 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
82 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
83 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
84 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
85 Train Loss 0.023872074 Test MSE 3.8414363198625523 Test RE 0.9991010183535421
86 Train Loss 0.023872074 Te

69 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
70 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
71 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
72 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
73 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
74 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
75 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
76 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
77 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
78 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
79 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
80 Train Loss 0.02382989 Test MSE 3.8404698695386643 Test RE 0.9989753306990248
81 Train Loss 0.02382989 Test MSE 3.8404

64 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
65 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
66 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
67 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
68 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
69 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
70 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
71 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
72 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
73 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
74 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
75 Train Loss 0.023829479 Test MSE 3.8403070165747226 Test RE 0.9989541499810756
76 Train Loss 0.023829479 Te

57 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
58 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
59 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
60 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
61 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
62 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
63 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
64 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
65 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
66 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
67 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
68 Train Loss 0.023823004 Test MSE 3.8394553640134608 Test RE 0.9988433764114555
69 Train Loss 0.023823004 Te

54 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
55 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
56 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
57 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
58 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
59 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
60 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
61 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
62 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
63 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
64 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
65 Train Loss 0.023853593 Test MSE 3.8414905235042256 Test RE 0.9991080671131795
66 Train Loss 0.023853593 Te

In [10]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)

0.9989821195084605
